In [1]:
import requests
import math
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [ ]:
url = 'https://www.century21.com/real-estate/new-orleans-la/LCLANEWORLEANS/'

driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)
time.sleep(2)
pages = driver.find_element_by_class_name('results-label')
pages = math.ceil(int(pages.text.split(' ')[-1][1:-1].replace(',',''))/100)
print(f'{pages} page will be processed.')

pages = range(1, pages+1)
data = []
for page in pages:
    surl = f'{url}?p={page}'
    driver.get(surl)
    driver.find_element_by_id('gallery-view-button').click()
    time.sleep(2)
    for i in range(7):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    all = soup.find_all('div', {'class': 'property-card'})
    for item in all:
        price = int(item.find(class_ = 'listing-price').text.strip().replace(',','').replace('$',''))
        city = item.find(class_ = 'property-city').text.strip().replace(',','').replace('$','')
        try:
            address = item.find(class_ = 'property-address').text.strip().replace(',','').replace('$','')
        except:
            address = None
        try:
            bed_rooms = item.find(class_ = 'property-beds').text.split()[0]
        except:
            bed_rooms = None        

        try:
            bath_rooms = item.find(class_ = 'property-baths').text.split()[0]
        except:
            bath_rooms = None

        try:
            sqft = int(item.find(class_ = 'property-sqft').text.split()[0].replace(',',''))
        except:
            sqft = None        

        try:
            status = item.find(class_ = 'property-image-flag sale-pending').text == 'Sale Pending'
        except:
            status = False          
        data_id = item['data-id']

        data.append({'city': city, 'address': address, 'bed_rooms': bed_rooms, 'bath_rooms': bath_rooms, 'area': sqft, 'price': price, 'pending_sale': status})
    print(f'PAGE: {page} ITEMS: {len(data)}')
driver.close()

df = pd.DataFrame(data)
df

9 page will be processed.
PAGE: 1 ITEMS: 100
PAGE: 2 ITEMS: 200
PAGE: 3 ITEMS: 300
PAGE: 4 ITEMS: 400
